### Forecasting Mini-Course Sales
スムージングさせた2020年データを活用  
https://www.kaggle.com/datasets/shivanimalhotra91/playground-s3e19-covid-data-smoothed  
国別係数も  
https://www.kaggle.com/code/aaachen/ps3e19-simple-eda-randomforest
- <b>学習は2021年を予測</b> (テスト予測には2021年も学習に使う?)
- 休日情報も使う
- 小さい値の追従性も鑑み、対数で学習させる
- ~~国、店舗、製品を分けて学習してみる~~
- ~~前年の売上げを特徴量として組み込んでみる~~

(plotly https://www.kaggle.com/code/mateuszk013/playground-series-s3e19-forecasting-sales)

In [ ]:
import numpy as np
import pandas as pd
from holidays import CountryHoliday
import warnings
warnings.simplefilter(action="ignore", category=FutureWarning)

: 

In [6]:
import plotly.express as px

FONT_COLOR = "#2F486B"
BACKGROUND_COLOR = "#FFFCFA"

In [26]:
import make_graph    # 自前
from bokeh.plotting import figure, output_notebook, show
output_notebook()

Loading BokehJS ...

In [4]:
df_train = pd.read_csv("./data/train.csv", parse_dates=["date"])
df_test = pd.read_csv("./data/test.csv", parse_dates=["date"])
df_subm = pd.read_csv("./data/sample_submission.csv")

In [31]:
# Thanks for shivanimalhotra91 providing for smoothed dataset.
# Reference: https://www.kaggle.com/datasets/shivanimalhotra91/playground-s3e19-covid-data-smoothed
df_train_smooth = pd.read_csv('./data/course_sales (1).csv', parse_dates=["date"])
df_train_smooth[df_train_smooth["date"]>="2020-01-01"].head()

,id,date,country,store,product,num_sold
82125,82125,2020-01-01,Argentina,Kaggle Learn,Using LLMs to Improve Your Coding,27.892596
82126,82126,2020-01-01,Argentina,Kaggle Learn,Using LLMs to Train More LLMs,28.609150
82127,82127,2020-01-01,Argentina,Kaggle Learn,Using LLMs to Win Friends and Influence People,3.792322
82128,82128,2020-01-01,Argentina,Kaggle Learn,Using LLMs to Win More Kaggle Competitions,27.413845
82129,82129,2020-01-01,Argentina,Kaggle Learn,Using LLMs to Write Better,21.181514


In [12]:
# plotly テスト

train_resampled_1d = df_train.resample("1D", on="date").mean(numeric_only=True)
train_rolling_7d = train_resampled_1d.rolling(window="7D").mean().reset_index()

fig = px.line(
    train_resampled_1d,
    y="num_sold",
    labels={"Num_Sold": "Mean Number of Sold Products"},
    title="Mean Number of Sold Products - Resampled to 1-Day Frequency",
    height=380,
    width=900,
    render_mode="webg1",
)
fig.update_traces(
    line=dict(width=1.0, color="#2F486B"),
    opacity=0.8,
)
fig.add_scatter(
    x=train_rolling_7d["date"],
    y=train_rolling_7d["num_sold"],
    name="Rolling Mean 7-Day Window",
    line=dict(width=1.5, color="#95424E"),
)
fig.update_layout(
    font_color=FONT_COLOR,
    title_font_size=18,
    plot_bgcolor=BACKGROUND_COLOR,
    paper_bgcolor=BACKGROUND_COLOR,
    xaxis_showgrid=False,
    yaxis_showgrid=False,
    xaxis_rangeslider_visible=True,
    legend=dict(yanchor="bottom", xanchor="right", y=1, x=1),
)
fig.add_vline(x="2020-03-01", line_width=2, line_dash="dash", line_color="#C94040")
fig.add_vline(x="2020-06-01", line_width=2, line_dash="dash", line_color="#C94040")
fig.add_annotation(x="2020-04-15", y=230, text="Covid Lockdown",
                   showarrow=False, font_size=14, textangle=-90)
fig.show()

In [33]:
CSP = {col: list(df_train[col].unique()) for col in ["country", "store", "product"]}

csp, df_buf = select_csp(df_train, [0, 0, 0])    # 関数は下にあります。。。
df_buf["kind"] = "orig"
csp, df_buf_s = select_csp(df_train_smooth, [0, 0, 0])
df_buf_s["kind"] = "smooth"
df_buf = pd.concat([df_buf, df_buf_s])
ttl = f"{[csp[k][0] for k in csp.keys() if len(csp[k]) < 2]}"
p = make_graph.make_trend(df_buf, "num_sold", "kind", 900, 300, ttl)
show(p)

↑　それっぽい。。2020年を置き換え

In [34]:
df_train["num_sold"] = df_train_smooth["num_sold"].copy()

In [38]:
# 一旦、学習用テスト用をまとめる
df_test["num_sold"] = np.nan
df_all = pd.concat([df_train, df_test], ignore_index=True)

df_all['year'] = df_all['date'].dt.year
df_all['month'] = df_all['date'].dt.month
df_all['day'] = df_all['date'].dt.day
print(df_all.shape)
df_all.head(5)

(164325, 9)


,id,date,country,store,product,num_sold,year,month,day
0,0,2017-01-01,Argentina,Kaggle Learn,Using LLMs to Improve Your Coding,63.0,2017,1,1
1,1,2017-01-01,Argentina,Kaggle Learn,Using LLMs to Train More LLMs,66.0,2017,1,1
2,2,2017-01-01,Argentina,Kaggle Learn,Using LLMs to Win Friends and Influence People,9.0,2017,1,1
3,3,2017-01-01,Argentina,Kaggle Learn,Using LLMs to Win More Kaggle Competitions,59.0,2017,1,1
4,4,2017-01-01,Argentina,Kaggle Learn,Using LLMs to Write Better,49.0,2017,1,1


In [36]:
# make_graph.trends(df_train, "num_sold", 1200, 300, "graph1_smooth.html", "origin")

In [39]:
CSP = {col: list(df_all[col].unique()) for col in ["country", "store", "product"]}
CSP

{'country': ['Argentina', 'Canada', 'Estonia', 'Japan', 'Spain'],
 'store': ['Kaggle Learn', 'Kaggle Store', 'Kagglazon'],
 'product': ['Using LLMs to Improve Your Coding',
  'Using LLMs to Train More LLMs',
  'Using LLMs to Win Friends and Influence People',
  'Using LLMs to Win More Kaggle Competitions',
  'Using LLMs to Write Better']}

In [40]:
def select_csp(df, idx):
    """インデックス指定で各名称取得、DataFrameフィルタ"""
    if len(idx) != 3:
        return (), pd.DataFrame()
    df_ret = df.copy()
    for i, col in zip(idx, ["country", "store", "product"]):
        if i < 0:
            continue
        idx_buf = i
        if i >= len(CSP[col]):
            print(f"[error] 指定インデックスが範囲外です。 ({col}: {CSP[col]})")
            idx_buf = 0
        df_ret = df_ret[df_ret[col] == CSP[col][idx_buf]]

    sel = {col: list(df_ret[col].unique()) for col in ["country", "store", "product"]}
    return sel, df_ret

In [41]:
# 出力テスト
csp, df_buf = select_csp(df_all, [1, 0, -1])
ttl = f"{[csp[k][0] for k in csp.keys() if len(csp[k]) < 2]}"
p = make_graph.make_trend(df_buf, "num_sold", "product", 900, 300, ttl)
show(p)

In [42]:
def make_dummies(df_temp, cat_cols):
    """国、店舗、製品情報をダミー変数化"""
    df_dummy = pd.get_dummies(df_temp[cat_cols]).astype(np.uint8)
    df_temp = pd.merge(df_temp, df_dummy, left_index=True, right_index=True)
    print("dummies:", df_temp.shape)
    return df_temp

In [43]:
def datetime_features(df_temp):
    """時間情報の特徴量生成"""
    df_temp['dayofyear'] = df_temp['date'].dt.day_of_year
    df_temp['dayofweek'] = df_temp['date'].dt.dayofweek
    df_temp['quarter'] = df_temp['date'].dt.quarter
    # df_temp['weekofyear'] = df_temp['date'].dt.weekofyear  # 見つからない
    df_temp['weekofyear'] = df_temp['date'].apply(lambda x: x.isocalendar()[1])
    df_temp['is_weekend'] = (df_temp['dayofweek'] >= 5).astype(np.int8)
    df_temp['is_month_start'] = df_temp['date'].dt.is_month_start.astype(np.int8)
    df_temp['is_month_end'] = df_temp['date'].dt.is_month_end.astype(np.int8)
    df_temp['dayname'] = df_temp['date'].dt.strftime("%A")    # 曜日。後でダミー変数化
    df_temp['is_quarter_end'] = df_temp['date'].dt.is_quarter_end.astype(np.uint8)
    df_temp['is_quarter_start'] = df_temp['date'].dt.is_quarter_start.astype(np.uint8)
    df_temp['is_year_end'] = df_temp['date'].dt.is_year_end.astype(np.uint8)
    df_temp['is_year_start'] = df_temp['date'].dt.is_year_start.astype(np.uint8)    
    print("datetime:", df_temp.shape)
    return df_temp

In [44]:
def GetAndAddHolidays(df):
    # Get country-specific holidays
    country_list = ['Argentina', 'Canada', 'Estonia', 'Japan', 'Spain']
    years = np.arange(df['year'].min(), df['year'].max() + 1)
    holidays = pd.DataFrame(columns=['date', 'holiday', 'country'])

    for country in country_list:
        for h in CountryHoliday(country, years=years).items():
            i=len(holidays)
            holidays.loc[i,'date']=pd.to_datetime(h[0])
            holidays.loc[i,'holiday']=h[1]
            holidays.loc[i,'country']=country

    # Add common holidays from Dec 24 to Dec 31 for each country
    common_holidays = pd.DataFrame({
    'date': pd.to_datetime(['2017-04-16','2017-12-24', '2017-12-25', '2017-12-26', '2017-12-27', '2017-12-28', '2017-12-29', '2017-12-30', '2017-12-31',
                            '2018-04-01','2018-12-24', '2018-12-25', '2018-12-26', '2018-12-27', '2018-12-28', '2018-12-29', '2018-12-30', '2018-12-31',
                            '2019-04-21','2019-12-24', '2019-12-25', '2019-12-26', '2019-12-27', '2019-12-28', '2019-12-29', '2019-12-30', '2019-12-31',
                            '2020-04-12','2020-12-24', '2020-12-25', '2020-12-26', '2020-12-27', '2020-12-28', '2020-12-29', '2020-12-30', '2020-12-31',
                            '2021-04-04','2021-12-24', '2021-12-25', '2021-12-26', '2021-12-27', '2021-12-28', '2021-12-29', '2021-12-30', '2021-12-31',
                            '2022-04-17','2022-12-24', '2022-12-25', '2022-12-26', '2022-12-27', '2022-12-28', '2022-12-29', '2022-12-30', '2022-12-31']),
    'holiday': 'Holiday Season'
    })

    country_common_holidays = pd.DataFrame()
    for country in country_list:
        temp_common_holidays = common_holidays.copy()
        temp_common_holidays['country'] = country
        country_common_holidays = pd.concat([country_common_holidays, temp_common_holidays], ignore_index=True)

    holidays = pd.concat([holidays, country_common_holidays], ignore_index=True)
    holidays['date'] = pd.to_datetime(holidays['date'])
    holidays['isHoliday']=1
    
    
    # Merge holidays with the original DataFrame based on date and country
    df = pd.merge(df, holidays, how='left', on=['date', 'country'])
    df['holiday'] = df['holiday'].fillna('Not Holiday')
    df['isHoliday'] = df['isHoliday'].fillna(0)
    
    # Add Weekend columns
    #df['Friday'] = (df['dayofweek'] == 4).astype(int)
    #df['Saturday'] = (df['dayofweek'] == 5).astype(int)
    #df['Sunday'] = (df['dayofweek'] == 6).astype(int)
    
    #Quarter, weekly season
    #df['Q1'] = ((df['month'] >= 1) & (df['month'] <= 3)).astype(int)
    #df['Q2'] = ((df['month'] >= 4) & (df['month'] <= 6)).astype(int)
    #df['Q3'] = ((df['month'] >= 7) & (df['month'] <= 9)).astype(int)
    #df['Q4'] = ((df['month'] >= 10) & (df['month'] <= 12)).astype(int)

    # Remove duplicates from df
    df = df.drop_duplicates(subset=['date', 'country','product','store'], keep='first').reset_index().drop(columns = ["index"])

    print("holidays:", df.shape)
    return df, holidays

#full, Holidays = GetAndAddHolidays(full)

In [45]:
def seasonality_features(df_temp):
    df_temp['month_sin'] = np.sin(2*np.pi*df_temp["month"]/12)
    df_temp['month_cos'] = np.cos(2*np.pi*df_temp["month"]/12)
    df_temp['day_sin'] = np.sin(2*np.pi*df_temp["day"]/24)    # 24 -> 30 -> 24
    df_temp['day_cos'] = np.cos(2*np.pi*df_temp["day"]/24)
    print("seasonality:", df_temp.shape)
    return df_temp

In [46]:
def make_features(df_temp):
    """全特徴量を追加"""
    df_temp = datetime_features(df_temp)
    df_temp, TrainHolidays = GetAndAddHolidays(df_all)
    df_temp = make_dummies(df_temp, ["country", "store", "product", "dayname", "holiday"])
    df_temp = seasonality_features(df_temp)
    return df_temp

# 特徴量生成
df_feat = make_features(df_all)
df_feat.head(3)

datetime: (164325, 21)
holidays: (164325, 23)
dummies: (164325, 129)
seasonality: (164325, 133)


,id,date,country,store,product,num_sold,year,month,day,dayofyear,...,holiday_敬老の日,holiday_文化の日,holiday_春分の日,holiday_昭和の日,holiday_海の日,holiday_秋分の日,month_sin,month_cos,day_sin,day_cos
0,0,2017-01-01,Argentina,Kaggle Learn,Using LLMs to Improve Your Coding,63.0,2017,1,1,1,...,0,0,0,0,0,0,0.5,0.866025,0.258819,0.965926
1,1,2017-01-01,Argentina,Kaggle Learn,Using LLMs to Train More LLMs,66.0,2017,1,1,1,...,0,0,0,0,0,0,0.5,0.866025,0.258819,0.965926
2,2,2017-01-01,Argentina,Kaggle Learn,Using LLMs to Win Friends and Influence People,9.0,2017,1,1,1,...,0,0,0,0,0,0,0.5,0.866025,0.258819,0.965926


各モデルの性能確認

In [47]:
import time
from sklearn.tree import DecisionTreeRegressor, plot_tree
from sklearn.preprocessing import MinMaxScaler, StandardScaler, LabelEncoder
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import Ridge, RidgeCV, Lasso, LassoCV, LinearRegression, ElasticNet
from sklearn.model_selection import KFold, StratifiedKFold, train_test_split, GridSearchCV, RepeatedKFold, RepeatedStratifiedKFold, ShuffleSplit, cross_val_score
from sklearn.metrics import explained_variance_score, mean_squared_error, mean_absolute_error, r2_score
from sklearn.inspection import PartialDependenceDisplay
from sklearn.ensemble import RandomForestRegressor, HistGradientBoostingRegressor, GradientBoostingRegressor, ExtraTreesRegressor
from sklearn.svm import SVR
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from catboost import CatBoostRegressor

In [48]:
def get_smape(actual, forecast):
    """自前で評価値(SMAPE:https://en.wikipedia.org/wiki/Symmetric_mean_absolute_percentage_error)産出"""
    smape = 100/len(actual) * np.sum(2 * np.abs(forecast - actual) / (np.abs(forecast) + np.abs(actual)))
    # smape = np.sum(np.abs(forecast - actual)) / np.sum(forecast + actual)
    return smape

学習用、検証用、テスト用にデータ分離 ~~(2020年を削除)~~

In [49]:
# 学習と検証用に分離 (2020年削除)
remove_cols = ["id", "date", "country", "store", "product", "num_sold", "dayname", "holiday"]
train = df_feat[df_feat["year"]<2021].copy()
valid = df_feat[df_feat["year"]==2021].copy()

X_train = train.drop(columns=remove_cols)
y_train = np.log(train["num_sold"].values)    # 小さい値のために対数で学習させる (後で戻す)
X_valid = valid.drop(columns=remove_cols)
y_valid = np.log(valid["num_sold"].values)
print(X_train.shape, len(y_train), X_valid.shape, len(y_valid))

(109575, 125) 109575 (27375, 125) 27375


In [50]:
# テスト用 (検証用の2021年も含める)
Xt_train = df_feat[~(df_feat["year"].isin([2022]))].drop(columns=remove_cols)
yt_train = np.log(df_feat[~(df_feat["year"].isin([2022]))]["num_sold"].values)
X_test = df_feat[df_feat["year"]==2022].drop(columns=remove_cols) 
print(Xt_train.shape, len(yt_train), X_test.shape)

(136950, 125) 136950 (27375, 125)


In [51]:
X_train.tail(3)

,year,month,day,dayofyear,dayofweek,quarter,weekofyear,is_weekend,is_month_start,is_month_end,...,holiday_敬老の日,holiday_文化の日,holiday_春分の日,holiday_昭和の日,holiday_海の日,holiday_秋分の日,month_sin,month_cos,day_sin,day_cos
109572,2020,12,31,366,3,4,53,0,0,1,...,0,0,0,0,0,0,-2.449294e-16,1.0,0.965926,-0.258819
109573,2020,12,31,366,3,4,53,0,0,1,...,0,0,0,0,0,0,-2.449294e-16,1.0,0.965926,-0.258819
109574,2020,12,31,366,3,4,53,0,0,1,...,0,0,0,0,0,0,-2.449294e-16,1.0,0.965926,-0.258819


In [52]:
yt_train

array([4.14313473, 4.18965474, 2.19722458, ..., 4.7095302 , 6.46302946,
       6.28971557])

In [53]:
regressors = [LinearRegression(), 
              Lasso(), 
              DecisionTreeRegressor(), 
              Ridge(), 
              ElasticNet(), 
              RandomForestRegressor(), 
              XGBRegressor(), 
              CatBoostRegressor(), 
              LGBMRegressor(), 
              GradientBoostingRegressor(), 
              HistGradientBoostingRegressor()]

In [54]:
scores = []
for model in regressors:
    name = str(model)[:-2]
    name = "XGBRegressor" if "XGBRegressor" in name else name
    name = "CatBoostRegressor" if "CatBoostRegressor" in name else name
    fit_param = {"X": X_train, "y": y_train}
    # if name in ["XGBRegressor", "CatBoostRegressor", "LGBMRegressor"]:
    #     fit_param["eval_set"] = [(X_valid, y_valid)]
    #     if name in ["CatBoostRegressor"]:
    #         fit_param["silent"] = True

    start = time.time()
    model.fit(**fit_param)
    train_time = time.time() - start
    y_pred = model.predict(X_valid)
    y_pred_exp_int = ((np.exp(y_pred) * 2 + 1) // 2).astype(int)    # 対数戻して、四捨五入int
    print(f"{name}, {train_time:.3f}s")
    scores.append([name, train_time, 
                   explained_variance_score(np.exp(y_valid), y_pred_exp_int),
                   mean_squared_error(np.exp(y_valid), y_pred_exp_int, squared=False), 
                   mean_absolute_error(np.exp(y_valid), y_pred_exp_int),
                   r2_score(np.exp(y_valid), y_pred_exp_int),
                   get_smape(np.exp(y_valid), y_pred_exp_int)])
df_score = pd.DataFrame(scores, columns=["model", "train_time", "explained_variance", "mae", "rmse", "r2", "smape"])

/tmp/ipykernel_5220/1470986145.py:16: RuntimeWarning:

overflow encountered in exp

/tmp/ipykernel_5220/1470986145.py:16: RuntimeWarning:

invalid value encountered in floor_divide

/tmp/ipykernel_5220/1470986145.py:16: RuntimeWarning:

invalid value encountered in cast



LinearRegression, 0.764s
Lasso, 0.115s
DecisionTreeRegressor, 1.040s
Ridge, 0.073s
ElasticNet, 0.118s
RandomForestRegressor, 66.269s
XGBRegressor, 6.659s
Learning rate set to 0.085991
0:	learn: 1.0584152	total: 74.2ms	remaining: 1m 14s
1:	learn: 0.9721042	total: 95.1ms	remaining: 47.4s
2:	learn: 0.8927441	total: 115ms	remaining: 38.1s
3:	learn: 0.8197456	total: 135ms	remaining: 33.6s
4:	learn: 0.7533449	total: 153ms	remaining: 30.5s
5:	learn: 0.6930004	total: 165ms	remaining: 27.3s
6:	learn: 0.6382281	total: 174ms	remaining: 24.7s
7:	learn: 0.5888377	total: 181ms	remaining: 22.5s
8:	learn: 0.5435691	total: 188ms	remaining: 20.6s
9:	learn: 0.5026270	total: 193ms	remaining: 19.1s
10:	learn: 0.4659015	total: 198ms	remaining: 17.8s
11:	learn: 0.4321973	total: 203ms	remaining: 16.7s
12:	learn: 0.4020904	total: 208ms	remaining: 15.8s
13:	learn: 0.3745799	total: 212ms	remaining: 14.9s
14:	learn: 0.3495838	total: 216ms	remaining: 14.2s
15:	learn: 0.3271004	total: 220ms	remaining: 13.5s
16:	lea

In [55]:
print("2020年含む")
df_score.sort_values("smape", ascending=True)

2020年含む


,model,train_time,explained_variance,mae,rmse,r2,smape
7,CatBoostRegressor,4.607187,9.784004e-01,3.148721e+01,1.640384e+01,9.748467e-01,9.144573
6,XGBRegressor,6.659250,9.784512e-01,3.143914e+01,1.644526e+01,9.749234e-01,9.246244
8,LGBMRegressor,0.448186,9.761221e-01,3.314650e+01,1.707719e+01,9.721258e-01,9.340393
10,HistGradientBoostingRegressor,2.132540,9.760783e-01,3.317642e+01,1.709107e+01,9.720755e-01,9.346965
5,RandomForestRegressor,66.269124,9.778809e-01,3.198571e+01,1.678670e+01,9.740439e-01,9.418005
2,DecisionTreeRegressor,1.040320,9.756827e-01,3.323709e+01,1.729878e+01,9.719732e-01,9.720372
9,GradientBoostingRegressor,14.903594,9.674304e-01,3.897767e+01,1.992117e+01,9.614558e-01,10.467900
0,LinearRegression,0.764458,-3.542012e+30,3.739546e+17,1.516171e+16,-3.547844e+30,13.550867
3,Ridge,0.072971,9.540416e-01,4.586001e+01,2.453680e+01,9.466426e-01,13.914314
1,Lasso,0.114841,0.000000e+00,2.161344e+02,1.278661e+02,-1.851534e-01,76.871744


In [87]:
df_score.sort_values("smape", ascending=True)

,model,train_time,explained_variance,mae,rmse,r2,smape
7,CatBoostRegressor,3.116341,9.783197e-01,3.220695e+01,1.722875e+01,9.736836e-01,9.965704
8,LGBMRegressor,0.264144,9.755026e-01,3.407647e+01,1.786663e+01,9.705398e-01,10.070713
10,HistGradientBoostingRegressor,1.638100,9.753132e-01,3.423111e+01,1.790327e+01,9.702718e-01,10.106009
6,XGBRegressor,4.162707,9.767978e-01,3.318206e+01,1.760354e+01,9.720660e-01,10.146105
5,RandomForestRegressor,47.512773,9.771875e-01,3.288201e+01,1.759872e+01,9.725689e-01,10.249666
9,GradientBoostingRegressor,10.469843,9.660201e-01,4.006379e+01,2.037958e+01,9.592778e-01,10.719073
2,DecisionTreeRegressor,0.767865,9.717171e-01,3.586826e+01,1.921863e+01,9.673602e-01,11.218350
0,LinearRegression,0.540498,-7.072360e+30,5.288517e+17,3.032341e+16,-7.095688e+30,14.402780
3,Ridge,0.052292,9.492418e-01,4.882348e+01,2.627317e+01,9.395238e-01,15.143136
1,Lasso,0.092458,0.000000e+00,2.157411e+02,1.277431e+02,-1.808441e-01,76.749820


In [56]:
def disp_scores(Y_test, test_pred):
    print("Test Scores")
    print("\tExplained variance:", explained_variance_score(Y_test, test_pred))
    print("\tMean absolute error:", mean_absolute_error(Y_test, test_pred))
    print("\tMean squared error:", mean_squared_error(Y_test, test_pred))
    print("\tR2 score:", r2_score(Y_test, test_pred))
    print("\tSMAPE score:", get_smape(Y_test, test_pred))

In [57]:
def get_prediction(train, valid, test, model):
    """各予測値を返す"""
    ret_arr = []
    for x in [train, valid, test]:
        prediction = model.predict(x)
        # # 小さい値で差分が出るのを抑えるため、四捨五入してみる    対数にしてたら、四捨五入ダメ
        # ret_arr.append(((prediction * 2 + 1) // 2).astype(int))
        ret_arr.append(prediction)
    return ret_arr

def learning(model, X_train, Y_train, X_valid, Y_valid, X):
    """学習、予測値出力"""
    model.fit(X_train, Y_train)
    train_pred, valid_pred, pred = get_prediction(X_train, X_valid, X, model)
    disp_scores(Y_valid, valid_pred)
    return (train_pred, valid_pred, pred)

In [58]:
def get_plot_dataframe(X_train, X_valid, train_pred, valid_pred, pred):
    """学習とテスト確認用 DataFrame 作成"""
    cols = ["id", "date", "country", "store", "product"]
    # オリジナル
    df_plt = df_train[cols].copy()
    df_plt["num_sold"] = df_train["num_sold"]
    df_plt["kind"] = "orig"
    # 学習結果
    df_train_pred = pd.merge(X_train, df_train, left_index=True, right_index=True)[cols]
    df_train_pred["num_sold"] = np.exp(train_pred)    # 対数で学習させてたら、指数で戻す
    df_train_pred["kind"] = "train"
    # 検証結果
    df_valid_pred = pd.merge(X_valid, df_train, left_index=True, right_index=True)[cols]
    df_valid_pred["num_sold"] = np.exp(valid_pred)
    df_valid_pred["kind"] = "test"
    # 実際の予測
    df_pred = df_test.copy()
    df_pred["num_sold"] = np.exp(pred)
    df_pred["kind"] = "pred"

    df_plt = pd.concat([df_plt, df_train_pred, df_valid_pred, df_pred]).reset_index(drop=True)
    # print(df_plt.shape)
    return df_plt

In [123]:
## RandomForest
model_name = "RandomForest"
model = RandomForestRegressor()
train_pred, valid_pred, pred = learning(model, X_train, y_train, X_valid, y_valid, X_test)
df_plt = get_plot_dataframe(X_train, X_valid, train_pred, valid_pred, pred)
csp, df_buf = select_csp(df_plt, [0, 0, 0])
ttl = f"{[csp[k][0] for k in csp.keys() if len(csp[k]) < 2]} ({model_name})"
p = make_graph.make_trend(df_buf, "num_sold", "kind", 900, 300, ttl)
show(p)

Test Scores
	Explained variance: 0.9926548603850557
	Mean absolute error: 0.10279542952759994
	Mean squared error: 0.016610505576286485
	R2 score: 0.9879665416451708
	SMAPE score: 2.5093895830919326


In [59]:
def check_correlation(df_plt):
    """国、店舗、製品毎の相関確認"""
    df_buf = df_plt[df_plt["kind"]=="test"].drop(columns=["kind"])
    df_buf.rename(columns={"num_sold": "num_sold_test"}, inplace=True)
    df_buf = pd.merge(df_buf, df_plt[df_plt["kind"]=="orig"].drop(columns=["kind"]))

    buf = []
    for c in df_buf["country"].unique():
        for s in df_buf["store"].unique():
            for p in df_buf["product"].unique():
                df_p = df_buf[(df_buf["country"]==c) & (df_buf["store"]==s) & (df_buf["product"]==p)]
                rmse = mean_squared_error(df_p["num_sold"], df_p["num_sold_test"], squared=False)
                smape = get_smape(df_p["num_sold"], df_p["num_sold_test"])
                buf.append([c, s, p, rmse, smape])
    df_ret = pd.DataFrame(buf, columns=["country", "store", "product", "rmse", "smape"])
    return df_ret

In [37]:
check_correlation(df_plt).sort_values("smape")

,country,store,product,rmse,smape
68,Spain,Kaggle Store,Using LLMs to Win More Kaggle Competitions,7.107203,4.915174
70,Spain,Kagglazon,Using LLMs to Improve Your Coding,30.800685,4.957550
73,Spain,Kagglazon,Using LLMs to Win More Kaggle Competitions,26.235812,4.999434
50,Japan,Kaggle Store,Using LLMs to Improve Your Coding,11.268090,5.034321
63,Spain,Kaggle Learn,Using LLMs to Win More Kaggle Competitions,4.825162,5.038845
...,...,...,...,...,...
31,Estonia,Kaggle Learn,Using LLMs to Train More LLMs,13.931928,18.548533
30,Estonia,Kaggle Learn,Using LLMs to Improve Your Coding,14.624337,18.769614
34,Estonia,Kaggle Learn,Using LLMs to Write Better,11.188668,18.815640
41,Estonia,Kagglazon,Using LLMs to Train More LLMs,78.860133,18.893183


In [90]:
## LightGBM
model_name = "LightGBM"
model = LGBMRegressor()
# model = LGBMRegressor(data_sample_strategy='goss', max_depth=15, n_estimators=150, num_leaves=31, force_row_wise=True)    # warning 抑制
train_pred, valid_pred, pred = learning(model, X_train, y_train, X_valid, y_valid, X_test)
df_plt = get_plot_dataframe(X_train, X_valid, train_pred, valid_pred, pred)
csp, df_buf = select_csp(df_plt, [0, 0, 0])
ttl = f"{[csp[k][0] for k in csp.keys() if len(csp[k]) < 2]} ({model_name})"
p = make_graph.make_trend(df_buf, "num_sold", "kind", 900, 300, ttl)
show(p)

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.003696 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 627
[LightGBM] [Info] Number of data points in the train set: 82125, number of used features: 107
[LightGBM] [Info] Start training from score 4.539447
Test Scores
	Explained variance: 0.9928228147152496
	Mean absolute error: 0.10155290538313574
	Mean squared error: 0.016158934870384226
	R2 score: 0.9882936814338292
	SMAPE score: 2.4497572565937524


In [35]:
df_corr = check_correlation(df_plt)
pd.concat([df_corr.sort_values("smape").head(3), df_corr.sort_values("smape").tail(3)])

,country,store,product,rmse,smape
45,Japan,Kaggle Learn,Using LLMs to Improve Your Coding,6.697342,4.794616
70,Spain,Kagglazon,Using LLMs to Improve Your Coding,31.724727,4.829026
71,Spain,Kagglazon,Using LLMs to Train More LLMs,30.923053,4.831571
34,Estonia,Kaggle Learn,Using LLMs to Write Better,11.480270,19.592509
41,Estonia,Kagglazon,Using LLMs to Train More LLMs,81.412668,19.676335
37,Estonia,Kaggle Store,Using LLMs to Win Friends and Influence People,3.517119,20.324771


In [36]:
csp, df_buf = select_csp(df_plt, [2, 1, 2])
ttl = f"{[csp[k][0] for k in csp.keys() if len(csp[k]) < 2]} ({model_name})"
p = make_graph.make_trend(df_buf, "num_sold", "kind", 900, 300, ttl)
show(p)

In [37]:
df_corr.groupby(by=["country"])["smape"].mean()

country
Argentina     8.572729
Canada       12.789617
Estonia      18.635249
Japan         5.247979
Spain         5.267670
Name: smape, dtype: float64

In [61]:
## Category Boosting
model_name = "CatBoost"
cat_params = {
    'n_estimators': 195,
    'learning_rate': 0.07725732658711602,
    'depth': 7,
    'l2_leaf_reg': 8.601133541582584,
    'subsample': 0.4279526734063217,
    'colsample_bylevel': 0.6767696482697301,
    'random_state': 42,
    'verbose': False
}
model = CatBoostRegressor(**cat_params)
train_pred, valid_pred, pred = learning(model, X_train, y_train, X_valid, y_valid, X_test)
df_plt = get_plot_dataframe(X_train, X_valid, train_pred, valid_pred, pred)
csp, df_buf = select_csp(df_plt, [0, 0, 0])
ttl = f"{[csp[k][0] for k in csp.keys() if len(csp[k]) < 2]} ({model_name})"
p = make_graph.make_trend(df_buf, "num_sold", "kind", 900, 300, ttl)
show(p)

Test Scores
	Explained variance: 0.9928948445114353
	Mean absolute error: 0.0929877325258514
	Mean squared error: 0.013960074405419406
	R2 score: 0.9898866429311005
	SMAPE score: 2.2227709437348646


In [62]:
## Category Boosting
model_name = "CatBoost"
cat_params = {'verbose':False}
model = CatBoostRegressor(**cat_params)
train_pred, valid_pred, pred = learning(model, X_train, y_train, X_valid, y_valid, X_test)
df_plt = get_plot_dataframe(X_train, X_valid, train_pred, valid_pred, pred)
csp, df_buf = select_csp(df_plt, [0, 0, 0])
ttl = f"{[csp[k][0] for k in csp.keys() if len(csp[k]) < 2]} ({model_name})"
p = make_graph.make_trend(df_buf, "num_sold", "kind", 900, 300, ttl)
show(p)

Test Scores
	Explained variance: 0.9929063495915406
	Mean absolute error: 0.09238397439842505
	Mean squared error: 0.013747894332095908
	R2 score: 0.9900403564989589
	SMAPE score: 2.220412590678514


In [63]:
df_corr = check_correlation(df_plt)
pd.concat([df_corr.sort_values("smape").head(3), df_corr.sort_values("smape").tail(3)])

,country,store,product,rmse,smape
70,Spain,Kagglazon,Using LLMs to Improve Your Coding,28.825009,4.508734
74,Spain,Kagglazon,Using LLMs to Write Better,21.344410,4.582037
73,Spain,Kagglazon,Using LLMs to Win More Kaggle Competitions,23.818569,4.587458
35,Estonia,Kaggle Store,Using LLMs to Improve Your Coding,21.773940,18.211769
37,Estonia,Kaggle Store,Using LLMs to Win Friends and Influence People,3.306838,18.251141
30,Estonia,Kaggle Learn,Using LLMs to Improve Your Coding,14.353481,18.365176


In [68]:
csp, df_buf = select_csp(df_plt, [2, 0, 0])
# csp, df_buf = select_csp(df_plt, [2, 0, 2])
ttl = f"{[csp[k][0] for k in csp.keys() if len(csp[k]) < 2]} ({model_name})"
p = make_graph.make_trend(df_buf, "num_sold", "kind", 900, 300, ttl)
show(p)

In [69]:
df_corr.groupby(by=["country"])["smape"].mean()

country
Argentina     6.509046
Canada       12.092770
Estonia      17.560946
Japan         5.110106
Spain         4.811317
Name: smape, dtype: float64

In [83]:
# valid が少しズレているので、国別に割合産出
for c in CSP["country"]:
    orig_country = df_train[(df_train["date"]>="2021-01-01") & (df_train["country"]==c)]["num_sold"]
    valid_pred_country = valid_pred[X_valid[f"country_{c}"]==1]    # 対数のままだった
    print(c, np.mean(orig_country / np.exp(valid_pred_country)))

Argentina 1.0248028660627184
Canada 1.1298849399792288
Estonia 1.1951616917360677
Japan 0.9850183051216217
Spain 1.0160533360469555


---
### submit
- 2021年まで含んでモデル生成

In [85]:
print(f'X_train:  {X_train["year"].unique()} \nXt_train: {Xt_train["year"].unique()}')

X_train:  [2017 2018 2019 2020] 
Xt_train: [2017 2018 2019 2020 2021]


In [86]:
print(model_name)
train_pred, valid_pred, pred = learning(model, Xt_train, yt_train, Xt_train, yt_train, X_test)
print(len(train_pred), len(valid_pred), len(pred))

CatBoost
Test Scores
	Explained variance: 0.9981783752951715
	Mean absolute error: 0.037808626544263334
	Mean squared error: 0.0024409734763965515
	R2 score: 0.998178375295162
	SMAPE score: 0.9450983279486651
136950 136950 27375


COVID-19 回復率反映 (！そのままの2020年使っていた場合)　スムーズ2020使っていたら不要

In [93]:
# # 2022年の倍率予測のため、2020年以前と2020年と2021年の比率を確認
# bef = df_all[df_all["year"]<2020].groupby(by=["country", "store", "product"])["num_sold"].median().rename("bef")
# cov = df_all[df_all["year"]==2020].groupby(by=["country", "store", "product"])["num_sold"].median().rename("cov")
# aft = df_all[df_all["year"]==2021].groupby(by=["country", "store", "product"])["num_sold"].median().rename("aft")
# df_buf = pd.concat([bef, cov, aft], axis=1).reset_index()
# df_buf["bef_ratio"] = df_buf["cov"] / df_buf["bef"]
# df_buf["aft_ratio"] = df_buf["aft"] / df_buf["cov"]

In [94]:
# df_country_ratio = df_buf.groupby("country")["aft_ratio"].mean().reset_index()
# df_country_ratio

,country,aft_ratio
0,Argentina,1.322779
1,Canada,1.276801
2,Estonia,1.249566
3,Japan,1.045555
4,Spain,1.184186


In [91]:
# 対数を戻して、国毎の倍率反映
# From https://www.kaggle.com/code/aaachen/ps3e19-simple-eda-randomforest
# coef_c = {'Argentina': 4.23, 'Spain': 1.500, 'Japan': 1.14, 'Estonia': 1.62, 'Canada': 0.87}
coef_c = {'Argentina': 3.372, 'Spain': 1.600, 'Japan': 1.394, 'Estonia': 1.651, 'Canada': 0.850}

pred_exp = np.exp(pred)
for country in coef_c.keys():
    pred_exp[df_test["country"]==country] *= coef_c[country]

# 四捨五入
pred_exp_int = ((pred_exp * 2 + 1) // 2).astype(int)

In [95]:
np.exp(np.array([v for v in coef_c.values()]))

array([29.13674231,  4.95303242,  4.03094161,  5.21218941,  2.33964685])

In [96]:
# # 更に掛ける (何故かスコアが良くなる。。。)
# # covid_coef = 1.2    # score:35.40569 (catboost)
# # covid_coef = 1.5    # やり過ぎ score:39.94571 (catboost)
# covid_coef = 1.3    # score:34.66034 (lgbm)
# pred_exp_int = pred_exp_int * covid_coef

In [96]:
df_plt = df_train.copy()
df_plt["kind"] = "origin"

df_buf = Xt_train.copy()
df_buf["num_sold"] = np.exp(train_pred)    # 2020年無いので、一旦Xt_trainへ移す
df_buf = df_train.drop(columns=["num_sold"]).join(df_buf[["num_sold"]], how="inner")
df_buf["kind"] = "train"
df_plt = pd.concat([df_plt, df_buf[df_plt.columns]])

df_buf = df_test.copy()
df_buf["num_sold"] = pred_exp_int
df_buf["kind"] = "pred"
df_plt = pd.concat([df_plt, df_buf[df_plt.columns]])

In [97]:
csp, df_buf = select_csp(df_plt, [0, 0, 0])
# csp, df_buf = select_csp(df_plt, [4, 0, 0])
ttl = f"{[csp[k][0] for k in csp.keys() if len(csp[k]) < 2]} ({model_name})"
p = make_graph.make_trend(df_buf, "num_sold", "kind", 900, 300, ttl)
show(p)

In [98]:
# df_subm = pd.merge(df_subm.drop(columns=["num_sold"]), df_test[["id", "num_sold"]].copy(), how="left")
df_subm["num_sold"] = pred_exp_int
print("num of Nan:", df_subm["num_sold"].isna().sum())
df_subm.describe()["num_sold"]

num of Nan: 0


count    27375.000000
mean       255.201973
std        245.328937
min         12.000000
25%        101.000000
50%        143.000000
75%        425.500000
max       1232.000000
Name: num_sold, dtype: float64

In [99]:
df_subm

,id,num_sold
0,136950,126
1,136951,123
2,136952,18
3,136953,119
4,136954,96
...,...,...
27370,164320,1186
27371,164321,1159
27372,164322,186
27373,164323,1104


In [100]:
df_subm.to_csv("./submission.csv", index=False)

`Score: 13.26282` 倍率高過ぎとは思ったけど、大幅改善。2020年のスムージングも効いたのかも知れないけど。